* 11:45 pm seems good
* check code and make sure no chains are hard coded into the url retrievals

In [2]:
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
from datetime import datetime
llama = DefiLlama()
import datetime as dt
import pytz
import sqlite3
from datetime import datetime, timezone

# flows = 2.5 seconds
# stables = half a min to a minute and a half
# tvl = 1 second
# dex vol = 

In [23]:
def get_most_recent_net_flows(chain_name: str):
    ids = []
    aggregated_data = {}
    tot = 0 
    # Fetch bridge data
    bridges_url = "https://bridges.llama.fi/bridges?includeChains=true"
    bridges_response = requests.get(bridges_url).json()
    
    # Collect IDs for bridges that involve the specified chain
    for bridge in bridges_response['bridges']:
        if chain_name in bridge['chains']:
            ids.append(bridge['id'])
    
    # Initialize variables to track the most recent date and its net flows
    
    for id in ids:
        url = f"https://bridges.llama.fi/bridgevolume/{chain_name}?id={id}"  # Use f-string for dynamic URL
        response = requests.get(url)
        if response.status_code == 200:  # Check if the request was successful
            d = response.json()
            if d:  # Check if the list is not empty
                d = d[-1]  # Get the most recent data
                # if the date of the most recent entry is = todays date or something like that
                tot += d['depositUSD'] - d['withdrawUSD']
            else:
                pass
        else:
            pass
    
    # Return the most recent date and its aggregated net flows
    aggregated_data[datetime.now().strftime('%Y-%m-%d')] = round(tot,2) 
    return aggregated_data

In [ ]:
today = datetime.now().strftime('%Y-%m-%d')

ids = []
bridges_url = "https://bridges.llama.fi/bridges?includeChains=true"
bridges_response = requests.get(bridges_url).json()

for bridge in bridges_response['bridges']:
    if 'Base' in bridge['chains']:
        ids.append(bridge['id'])

for id in ids:
    url = f"https://bridges.llama.fi/bridgevolume/Ethereum?id={id}"

In [4]:
def get_most_recent_stables(chain_name: str):
    stable_ids = list(range(1, 161))
    total = 0
    date = None  # Initialize date variable outside the loop

    for id in stable_ids:
        url = f"https://stablecoins.llama.fi/stablecoincharts/{chain_name}?stablecoin={id}"
        response = requests.get(url).json()
        if response:  # Ensure the response is not empty
            last_entry = response[-1]
            total += last_entry.get('totalCirculating', {}).get('peggedUSD', 0)
            date = datetime.utcfromtimestamp(int(last_entry['date'])).replace(tzinfo=pytz.utc).astimezone(pytz.timezone("US/Eastern"))

    if date:  # Check if date was updated (implies at least one successful data fetch)
        return {date.strftime('%Y-%m-%d'): total}
    else:
        return {}


In [5]:
def get_most_recent_tvl(chain_name: str):
    """
    Fetches the most recent Total Value Locked (TVL) for the specified chain.
    """
    tvl_url = f"https://api.llama.fi/v2/historicalChainTvl/{chain_name}"
    response = requests.get(tvl_url)
    if response.status_code == 200:
        tvl_data = response.json()
        if tvl_data:
            # Assuming the last entry is the most recent
            most_recent_entry = tvl_data[-1]
            date = datetime.utcfromtimestamp(int(most_recent_entry['date'])).replace(tzinfo=pytz.utc).astimezone(pytz.timezone("US/Eastern"))
            most_recent_date = date.strftime('%Y-%m-%d')
            most_recent_tvl = most_recent_entry['tvl']
            
            # Return the most recent date and its TVL
            return {most_recent_date: most_recent_tvl}
    return {}

In [6]:
def get_dex_vol(chain_name:str):
    """
    Get aggregate dex volume done on each chain aggregated across all dexes
    """
    dex_volume = {'24H': 0, '7D': 0, '30D': 0}
    dex_url = f"https://api.llama.fi/overview/dexs/{chain_name}?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"

    try:
        dex_data = requests.get(dex_url).json()
        for entry in dex_data['protocols']:
            dex_volume['24H'] += entry.get('total24h', 0) if entry.get('total24h') is  not None else 0 
            dex_volume['7D'] += entry.get('total7d', 0) if entry.get('total7d') is  not None else 0 
            dex_volume['30D'] += entry.get('total30d', 0) if entry.get('total30d') is  not None else 0 
    except Exception as e:
        print(f"An error occurred: {e}")

    dex_volume['24H'] = round(dex_volume['24H'],2)
    dex_volume['7D'] = round(dex_volume['7D'],2)
    dex_volume['30D'] = round(dex_volume['30D'],2)

    return dex_volume


In [8]:
get_dex_vol('Base')

{'24H': 35739805, '7D': 238975946.63, '30D': 834756126.25}

### Connect back to the SQL databse with historical data, and update the most recent dates data:

In [7]:
def entry_exists(chain_name, date, table_name):
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    query = f"SELECT EXISTS(SELECT 1 FROM {table_name} WHERE chain_name=? AND date=? LIMIT 1)"
    cursor.execute(query, (chain_name, date))
    exists = cursor.fetchone()[0]
    
    conn.close()
    return exists == 1

def insert_data(table_name, data_dict):
    """
    Insert data into the specified table.
    
    Parameters:
    - table_name: The name of the table to insert data into.
    - data_dict: A dictionary where keys are column names and values are the data to insert.
    """
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Prepare column names and placeholders for values
    columns = ', '.join(data_dict.keys())
    placeholders = ', '.join(['?' for _ in data_dict])
    
    query = f'''INSERT OR REPLACE INTO {table_name} ({columns}) VALUES ({placeholders})'''
    cursor.execute(query, list(data_dict.values()))
    
    conn.commit()
    conn.close()

In [8]:
def fetch_and_insert_flows(chain_name):
    # Use the modified get_net_flows function that focuses on the most recent data
    flows_data = get_most_recent_net_flows(chain_name)
    for date, net_inflow in flows_data.items():
        if not entry_exists(chain_name, date, 'net_flows'):
            insert_data('net_flows', {
                'chain_name': chain_name,
                'date': date,
                'net_inflow': net_inflow
            })

def fetch_and_insert_stablecoins(chain_name):
    # Use the modified get_stables function
    stables_data = get_most_recent_stables(chain_name)
    for date, total_stables in stables_data.items():
        if not entry_exists(chain_name, date, 'Stablecoins'):
            insert_data('Stablecoins', {
                'chain_name': chain_name,
                'date': date,
                'total_stables': total_stables
            })

def fetch_and_insert_tvl(chain_name):
    # Use the modified get_tvl function
    tvl_data = get_most_recent_tvl(chain_name)
    for date, tvl in tvl_data.items():
        if not entry_exists(chain_name, date, 'TVL'):
            insert_data('TVL', {
                'chain_name': chain_name,
                'date': date,
                'TVL': tvl
            })

def fetch_and_insert_dex_volume(chain_name):
    dex_volume_data = get_dex_vol(chain_name)  # Ensure this function returns the most recent 24H, 7D, and 30D volume data
    # Prepare the data dictionary for insertion
    formatted_dex_volume_data = {
        'chain_name': chain_name,
        'volume_24h': dex_volume_data.get('24H', 0),
        'volume_7d': dex_volume_data.get('7D', 0),
        'volume_30d': dex_volume_data.get('30D', 0),
    }
    # Directly insert or replace the DEX volume data for the chain
    insert_data('Dex_Volume', formatted_dex_volume_data)



In [9]:
chains = ['Ethereum', 'Solana', 'Base', 'Sei', 'Sui', 'Injective', 'Avalanche', 'Optimism']

for chain in chains:
    fetch_and_insert_flows(chain)
    fetch_and_insert_stablecoins(chain)
    fetch_and_insert_tvl(chain)
    fetch_and_insert_dex_volume(chain)


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

## TO DO:
- Calculate % changes and stuff by chain for each data point by pulling from SQL
- Find a way to present it pretty like slurpxbt
- Find a way to automate the data pull + addition to db and email sending

In [10]:
# ETH, SOL, Base, Avalanche, Optimism
def query_database_flows():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM net_flows")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_flows()

('Ethereum', '2023-08-20', 34.0)
('Ethereum', '2023-08-21', 0.0)
('Ethereum', '2023-08-22', 6741.0)
('Ethereum', '2023-08-23', 0.0)
('Ethereum', '2023-08-24', 0.0)
('Ethereum', '2023-08-25', 0.0)
('Ethereum', '2023-08-26', 0.0)
('Ethereum', '2023-08-27', 8039.0)
('Ethereum', '2023-08-28', 106.0)
('Ethereum', '2023-08-29', 100.0)
('Ethereum', '2023-08-30', 50.0)
('Ethereum', '2023-08-31', 34.0)
('Ethereum', '2023-09-01', 0.0)
('Ethereum', '2023-09-02', 0.0)
('Ethereum', '2023-09-03', 2357.0)
('Ethereum', '2023-09-04', 616765.0)
('Ethereum', '2023-09-05', 362160.0)
('Ethereum', '2023-09-06', 1113224.0)
('Ethereum', '2023-09-07', 125599.0)
('Ethereum', '2023-09-08', 102061.0)
('Ethereum', '2023-09-09', 112823.0)
('Ethereum', '2023-09-10', -71059.0)
('Ethereum', '2023-09-11', -132610.0)
('Ethereum', '2023-09-12', 300812.0)
('Ethereum', '2023-09-13', -279567.0)
('Ethereum', '2023-09-14', 508757.0)
('Ethereum', '2023-09-15', -180776.0)
('Ethereum', '2023-09-16', 296074.0)
('Ethereum', '2023-

In [11]:
# ETH, SOL, Base, Avalanche, Optimism, SUI
def query_database_stables():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM Stablecoins")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_stables()

('Ethereum', '2023-12-20', 101091632.75)
('Ethereum', '2023-12-21', 67167914390.89)
('Ethereum', '2023-12-22', 67428619534.49999)
('Ethereum', '2023-12-23', 67555975986.63997)
('Ethereum', '2023-12-24', 67605626716.739975)
('Ethereum', '2023-12-25', 67738494129.219986)
('Ethereum', '2023-12-26', 67672636951.189995)
('Ethereum', '2023-12-27', 67915582462.729996)
('Ethereum', '2023-12-28', 67937060189.37999)
('Ethereum', '2023-12-29', 67928559321.389984)
('Ethereum', '2023-12-30', 67904623094.16999)
('Ethereum', '2023-12-31', 67728359605.96999)
('Ethereum', '2024-01-01', 67862428530.43999)
('Ethereum', '2024-01-02', 68309424340.94002)
('Ethereum', '2024-01-03', 68531725971.29001)
('Ethereum', '2024-01-04', 68850964393.35002)
('Ethereum', '2024-01-05', 69291479990.37997)
('Ethereum', '2024-01-06', 69393582760.38998)
('Ethereum', '2024-01-07', 69556150920.29002)
('Ethereum', '2024-01-08', 69714655088.26003)
('Ethereum', '2024-01-09', 69734418809.99998)
('Ethereum', '2024-01-10', 6982550418

In [12]:
# ETH, SOL, Base, SEI, Sui, INJ, Avalanche, Optimism, 
def query_database_tvl():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM TVL")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_tvl()

('Ethereum', '2023-12-21', 29648949441.18832)
('Ethereum', '2023-12-22', 29497796341.495186)
('Ethereum', '2023-12-23', 29713190845.820255)
('Ethereum', '2023-12-24', 29619267620.200104)
('Ethereum', '2023-12-25', 29224549491.198788)
('Ethereum', '2023-12-26', 29606181699.653942)
('Ethereum', '2023-12-27', 30934738062.265)
('Ethereum', '2023-12-28', 30610489342.97283)
('Ethereum', '2023-12-29', 29751828881.071396)
('Ethereum', '2023-12-30', 30039788050.67762)
('Ethereum', '2023-12-31', 30107479147.252426)
('Ethereum', '2024-01-01', 31387922717.0552)
('Ethereum', '2024-01-02', 31139542136.891754)
('Ethereum', '2024-01-03', 29445658278.377575)
('Ethereum', '2024-01-04', 30306379703.53779)
('Ethereum', '2024-01-05', 30099527782.84863)
('Ethereum', '2024-01-06', 30137272840.94493)
('Ethereum', '2024-01-07', 29948398089.118916)
('Ethereum', '2024-01-08', 30598405979.523666)
('Ethereum', '2024-01-09', 32250960914.627888)
('Ethereum', '2024-01-10', 34734965861.052986)
('Ethereum', '2024-01-11

In [13]:
# ETH, SOL, Base, Avalanche, Optimism, SUI
def query_database_dex_volume():
    conn = sqlite3.connect('../chain_data.db')
    cursor = conn.cursor()
    
    # Example: Query the net_flows table
    cursor.execute("SELECT * FROM Dex_volume")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

query_database_dex_volume()

('Avalanche', 78180304.0, 682268981.0, 2437630631.6809998)
('Optimism', 14750983.0, 135408352.0, 495986335.94299996)
('Ethereum', 1525516266.0, 9480630349.0, 36100158135.28)
('Solana', 656141259.0, 4893316418.0, 21288972156.67)
('Base', 34601403.0, 220868367.0, 871110366.44)
('Sei', 1161393.0, 11728828.0, 37668142.89)
('Sui', 43776361.0, 430394902.0, 2460986431.06)
('Injective', 23602396.0, 155238977.0, 613525525.72)
